In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve,roc_auc_score, auc
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , precision_score , recall_score ,  f1_score , precision_recall_curve , roc_auc_score 
from sklearn.preprocessing import RobustScaler,StandardScaler
from xgboost import XGBClassifier

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('creditcard.csv')

In [4]:
data.drop_duplicates(inplace=True)

In [5]:
data.duplicated().sum()

np.int64(0)

In [6]:
X = data.drop(columns=['Class'])
y = data[['Class']]

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.75,random_state=42)

In [14]:
train = pd.concat([X_train,y_train],axis=1)

i have to get the balanced data from the train only :
- if i get frauds and non_frauds from the whole data then test on the ( X_test ) this will cause a data leakage

In [15]:
nor_frauds = train[train['Class'] == 0]   # trian not data
frauds = train[train['Class'] == 1]
nor_frauds['Class'].value_counts(),frauds['Class'].value_counts()

(Class
 0    212433
 Name: count, dtype: int64,
 Class
 1    361
 Name: count, dtype: int64)

In [21]:
balanced_df = pd.concat([ frauds , nor_frauds.sample(500*len(frauds) ,random_state=42)],axis=0)
balanced_df['Class'].value_counts()

Class
0    180500
1       361
Name: count, dtype: int64

In [22]:
x_balanced = balanced_df.drop(columns=['Class'])
y_balanced = balanced_df[['Class']]

In [23]:
x_bakanced_train ,x_bakanced_test , y_bakanced_train,y_bakanced_test = train_test_split(x_balanced,y_balanced,train_size=.80,stratify=y_balanced,random_state=42)

In [50]:
xgb_model = XGBClassifier(scale_pos_weight=14, random_state=42)
xgb_model.fit(x_bakanced_train, y_bakanced_train)
y_pred = xgb_model.predict(X_test)
y_proba = xgb_model.predict_proba(X_test)
print("XGB results : \n")
print(f"accuracy_score is : {accuracy_score(y_test, y_pred)} ")
print(f"precision_score is : {precision_score(y_test, y_pred)}")
print(f"recall_score is : {recall_score(y_test, y_pred)}")
print(f"f1_score is : {f1_score(y_test, y_pred)}")

precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba[:,1])

print("best threshold for max f1_score : \n")

# حساب F1 لكل threshold
f1_scores = 2 * (precisions * recalls) / (precisions + recalls)

# أفضل threshold
best_threshold = thresholds[f1_scores.argmax()]
print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")

print("best threshold for max recall : \n")
min_precision = .9
valid_idx = np.where(precisions[:-1] >= min_precision)[0]
best_threshold = thresholds[valid_idx][recalls[valid_idx].argmax()]

print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")

XGB results : 

accuracy_score is : 0.9992950995319461 
precision_score is : 0.7627118644067796
recall_score is : 0.8035714285714286
f1_score is : 0.782608695652174
best threshold for max f1_score : 

best Threshold: 0.9010685
Precision (custom threshold): 0.925531914893617
Recall (custom threshold): 0.7767857142857143
F1-score (custom threshold): 0.8446601941747572
best threshold for max recall : 

best Threshold: 0.867203
Precision (custom threshold): 0.90625
Recall (custom threshold): 0.7767857142857143
F1-score (custom threshold): 0.8365384615384616


In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(x_bakanced_train,y_bakanced_train)
y_predicted = model.predict(x_bakanced_test)

score_acc = accuracy_score(y_bakanced_test , y_predicted)
score_precision = precision_score(y_bakanced_test , y_predicted)
score_recall = recall_score(y_bakanced_test , y_predicted)
score_f1 = f1_score(y_bakanced_test , y_predicted)

print(f"Model: {model.__class__.__name__}")
print(f"Accuracy  : {score_acc:.4f}")
print(f"Precision : {score_precision:.4f}")
print(f"Recall    : {score_recall:.4f}")
print(f"F1 Score  : {score_f1:.4f}")
print("*"*30)

y_predicted = model.predict(X_test)
y_proba = model.predict_proba(X_test)
score_acc = accuracy_score(y_test , y_predicted)
score_precision = precision_score(y_test , y_predicted)
score_recall = recall_score(y_test , y_predicted)
score_f1 = f1_score(y_test , y_predicted)
print('real test results : ')
print(f"Model: {model.__class__.__name__}")
print(f"Accuracy  : {score_acc:.4f}")
print(f"Precision : {score_precision:.4f}")
print(f"Recall    : {score_recall:.4f}")
print(f"F1 Score  : {score_f1:.4f}")
print("*"*30)
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba[:,1])

print("best threshold for max f1_score : \n")

f1_scores = 2 * (precisions * recalls) / (precisions + recalls)

best_threshold = thresholds[f1_scores.argmax()]
print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")

print("best threshold for max recall : \n")
min_precision = .9
valid_idx = np.where(precisions[:-1] >= min_precision)[0]
best_threshold = thresholds[valid_idx][recalls[valid_idx].argmax()]

print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")

In [ ]:
random_forest = RandomForestClassifier(n_estimators = 200 , random_state = 42 , n_jobs = -1)
random_forest.fit(x_bakanced_train , y_bakanced_train)
y_pred = random_forest.predict(X_test)
y_proba = random_forest.predict_proba(X_test)
print(f"accuracy_score is : {accuracy_score(y_test, y_pred)} ")
print(f"precision_score is : {precision_score(y_test, y_pred)}")
print(f"recall_score is : {recall_score(y_test, y_pred)}")
print(f"f1_score is : {f1_score(y_test, y_pred)}")
print(f"The ROC score is : {roc_auc_score(y_test , y_proba[:,1])}")

print("*" * 30)

precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba[:,1])

print("best threshold for max f1_score : \n")


f1_scores = 2 * (precisions * recalls) / (precisions + recalls)

best_threshold = thresholds[f1_scores.argmax()]
print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")

print("best threshold for max recall : \n")
min_precision = .9
valid_idx = np.where(precisions[:-1] >= min_precision)[0]
best_threshold = thresholds[valid_idx][recalls[valid_idx].argmax()]

print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")



accuracy_score is : 0.9995241955380115 
precision_score is : 0.8987341772151899
recall_score is : 0.7888888888888889
f1_score is : 0.8402366863905325
The ROC score is : 0.9454606161159748
******************************
best threshold for max f1_score : 

best Threshold: 0.535
Precision (custom threshold): 0.9102564102564102
Recall (custom threshold): 0.7888888888888889
F1-score (custom threshold): 0.8452380952380952
best threshold for max recall : 

best Threshold: 0.535
Precision (custom threshold): 0.9102564102564102
Recall (custom threshold): 0.7888888888888889
F1-score (custom threshold): 0.8452380952380952


In [9]:
from imblearn.combine import SMOTETomek

In [ ]:
os = SMOTETomek(sampling_strategy=0.5, random_state=42)
X_train_ns, y_train_ns = os.fit_resample(X_train, y_train)

In [ ]:
random_forest = RandomForestClassifier(n_estimators = 200 , random_state = 42 , n_jobs = -1)
random_forest.fit(X_train_ns,y_train_ns)
y_pred = random_forest.predict(X_test)
y_proba = random_forest.predict_proba(X_test)
print(f"accuracy_score is : {accuracy_score(y_test, y_pred)} ")
print(f"precision_score is : {precision_score(y_test, y_pred)}")
print(f"recall_score is : {recall_score(y_test, y_pred)}")
print(f"f1_score is : {f1_score(y_test, y_pred)}")
print(f"The ROC score is : {roc_auc_score(y_test , y_proba[:,1])}")

print("*" * 30)

precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba[:,1])

print("best threshold for max f1_score : \n")


f1_scores = 2 * (precisions * recalls) / (precisions + recalls)

best_threshold = thresholds[f1_scores.argmax()]
print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")

print("best threshold for max recall : \n")
min_precision = .9
valid_idx = np.where(precisions[:-1] >= min_precision)[0]
best_threshold = thresholds[valid_idx][recalls[valid_idx].argmax()]

print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")


accuracy_score is : 0.9995770597191677 
precision_score is : 0.9183673469387755
recall_score is : 0.8035714285714286
f1_score is : 0.8571428571428571
The ROC score is : 0.9725194280267884
******************************
best threshold for max f1_score : 

best Threshold: 0.54
Precision (custom threshold): 0.9278350515463918
Recall (custom threshold): 0.8035714285714286
F1-score (custom threshold): 0.861244019138756
best threshold for max recall : 

best Threshold: 0.435
Precision (custom threshold): 0.900990099009901
Recall (custom threshold): 0.8125
F1-score (custom threshold): 0.8544600938967136


Best results till now

In [10]:
os = SMOTETomek(sampling_strategy=0.75, random_state=42)
X_train_ns, y_train_ns = os.fit_resample(X_train, y_train)

In [11]:
random_forest = RandomForestClassifier(n_estimators = 200 , random_state = 42 , n_jobs = -1)
random_forest.fit(X_train_ns,y_train_ns)
y_pred = random_forest.predict(X_test)
y_proba = random_forest.predict_proba(X_test)
print(f"accuracy_score is : {accuracy_score(y_test, y_pred)} ")
print(f"precision_score is : {precision_score(y_test, y_pred)}")
print(f"recall_score is : {recall_score(y_test, y_pred)}")
print(f"f1_score is : {f1_score(y_test, y_pred)}")
print(f"The ROC score is : {roc_auc_score(y_test , y_proba[:,1])}")

print("*" * 30)

precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba[:,1])

print("best threshold for max f1_score : \n")


f1_scores = 2 * (precisions * recalls) / (precisions + recalls)

best_threshold = thresholds[f1_scores.argmax()]
print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")

print("best threshold for max recall : \n")
min_precision = .9
valid_idx = np.where(precisions[:-1] >= min_precision)[0]
best_threshold = thresholds[valid_idx][recalls[valid_idx].argmax()]

print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")


accuracy_score is : 0.9995629617098066 
precision_score is : 0.9090909090909091
recall_score is : 0.8035714285714286
f1_score is : 0.8530805687203792
The ROC score is : 0.9770366648848186
******************************
best threshold for max f1_score : 

best Threshold: 0.42
Precision (custom threshold): 0.9108910891089109
Recall (custom threshold): 0.8214285714285714
F1-score (custom threshold): 0.863849765258216
best threshold for max recall : 

best Threshold: 0.395
Precision (custom threshold): 0.9019607843137255
Recall (custom threshold): 0.8214285714285714
F1-score (custom threshold): 0.8598130841121495


best results

In [26]:
os = SMOTETomek(sampling_strategy=0.5, random_state=42)
X_balanced_train_ns, y_balanced_train_ns = os.fit_resample(x_bakanced_train, y_bakanced_train)

In [ ]:
balanced_df = pd.concat([ frauds , nor_frauds.sample(500*len(frauds) ,random_state=42)],axis=0)
balanced_df['Class'].value_counts()

Class
0    180500
1       361
Name: count, dtype: int64

In [ ]:
x_balanced = balanced_df.drop(columns=['Class'])
y_balanced = balanced_df[['Class']]

In [ ]:
x_bakanced_train ,x_bakanced_test , y_bakanced_train,y_bakanced_test = train_test_split(x_balanced,y_balanced,train_size=.80,stratify=y_balanced,random_state=42)

In [ ]:
os = SMOTETomek(sampling_strategy=0.5, random_state=42)
X_balanced_train_ns, y_balanced_train_ns = os.fit_resample(x_bakanced_train, y_bakanced_train)

In [27]:
random_forest = RandomForestClassifier(n_estimators = 200 , random_state = 42 , n_jobs = -1)
random_forest.fit(X_balanced_train_ns,y_balanced_train_ns)
y_pred = random_forest.predict(X_test)
y_proba = random_forest.predict_proba(X_test)
print(f"accuracy_score is : {accuracy_score(y_test, y_pred)} ")
print(f"precision_score is : {precision_score(y_test, y_pred)}")
print(f"recall_score is : {recall_score(y_test, y_pred)}")
print(f"f1_score is : {f1_score(y_test, y_pred)}")
print(f"The ROC score is : {roc_auc_score(y_test , y_proba[:,1])}")

print("*" * 30)

precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba[:,1])

print("best threshold for max f1_score : \n")


f1_scores = 2 * (precisions * recalls) / (precisions + recalls)

best_threshold = thresholds[f1_scores.argmax()]
print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")

print("best threshold for max recall : \n")
min_precision = .9
valid_idx = np.where(precisions[:-1] >= min_precision)[0]
best_threshold = thresholds[valid_idx][recalls[valid_idx].argmax()]

print("best Threshold:", best_threshold)
y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
print(f"Precision (custom threshold): {precision_score(y_test, y_pred_custom)}")
print(f"Recall (custom threshold): {recall_score(y_test, y_pred_custom)}")
print(f"F1-score (custom threshold): {f1_score(y_test, y_pred_custom)}")


accuracy_score is : 0.9995206676817233 
precision_score is : 0.89
recall_score is : 0.7946428571428571
f1_score is : 0.839622641509434
The ROC score is : 0.9641654395449227
******************************
best threshold for max f1_score : 

best Threshold: 0.68
Precision (custom threshold): 0.956989247311828
Recall (custom threshold): 0.7946428571428571
F1-score (custom threshold): 0.8682926829268293
best threshold for max recall : 

best Threshold: 0.555
Precision (custom threshold): 0.9081632653061225
Recall (custom threshold): 0.7946428571428571
F1-score (custom threshold): 0.8476190476190476


best results 

In [34]:
from sklearn.model_selection import KFold, cross_val_score

In [35]:
model = RandomForestClassifier(random_state=42)
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5 folds

scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
print("Scores for each fold:", scores)
print("Mean accuracy:", scores.mean())

Scores for each fold: [0.99954182 0.99964755 0.99943607 0.99968279 0.99948894]
Mean accuracy: 0.9995594341640042


In [37]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y), 1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]   # <--- FIXED
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]   # <--- FIXED

    # Resample ONLY training data
    smt = SMOTETomek(random_state=42)
    X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

    # Fit Random Forest
    random_forest = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
    random_forest.fit(X_train_res, y_train_res)

    # Predictions
    y_pred = random_forest.predict(X_test)
    y_proba = random_forest.predict_proba(X_test)

    # Basic metrics
    print(f"\n=== Fold {fold} ===")
    print(f"accuracy_score: {accuracy_score(y_test, y_pred):.4f}")
    print(f"precision_score: {precision_score(y_test, y_pred):.4f}")
    print(f"recall_score: {recall_score(y_test, y_pred):.4f}")
    print(f"f1_score: {f1_score(y_test, y_pred):.4f}")
    print(f"roc_auc_score: {roc_auc_score(y_test, y_proba[:,1]):.4f}")

    # Precision-recall curve
    precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba[:,1])
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls)

    # Best threshold (max F1)
    best_threshold = thresholds[f1_scores.argmax()]
    y_pred_custom = (y_proba[:,1] >= best_threshold).astype(int)
    print(f"Best threshold (max F1): {best_threshold:.4f}")
    print(f"Custom Precision: {precision_score(y_test, y_pred_custom):.4f}")
    print(f"Custom Recall: {recall_score(y_test, y_pred_custom):.4f}")
    print(f"Custom F1: {f1_score(y_test, y_pred_custom):.4f}")


=== Fold 1 ===
accuracy_score: 0.9995
precision_score: 0.9211
recall_score: 0.7778
f1_score: 0.8434
roc_auc_score: 0.9692
Best threshold (max F1): 0.6400
Custom Precision: 0.9589
Custom Recall: 0.7778
Custom F1: 0.8589

=== Fold 2 ===
accuracy_score: 0.9996
precision_score: 0.8537
recall_score: 0.8434
f1_score: 0.8485
roc_auc_score: 0.9900
Best threshold (max F1): 0.7850
Custom Precision: 0.9200
Custom Recall: 0.8313
Custom F1: 0.8734

=== Fold 3 ===
accuracy_score: 0.9994
precision_score: 0.8969
recall_score: 0.7699
f1_score: 0.8286
roc_auc_score: 0.9591
Best threshold (max F1): 0.6900
Custom Precision: 0.9247
Custom Recall: 0.7611
Custom F1: 0.8350

=== Fold 4 ===
accuracy_score: 0.9996
precision_score: 0.8587
recall_score: 0.8681
f1_score: 0.8634
roc_auc_score: 0.9768
Best threshold (max F1): 0.8550
Custom Precision: 0.9383
Custom Recall: 0.8352
Custom F1: 0.8837

=== Fold 5 ===
accuracy_score: 0.9995
precision_score: 0.8526
recall_score: 0.8438
f1_score: 0.8482
roc_auc_score: 0.98

In [38]:
# Store your fold results in lists
accuracy = [0.9995, 0.9996, 0.9994, 0.9996, 0.9995]
precision = [0.9211, 0.8537, 0.8969, 0.8587, 0.8526]
recall = [0.7778, 0.8434, 0.7699, 0.8681, 0.8438]
f1 = [0.8434, 0.8485, 0.8286, 0.8634, 0.8482]
roc_auc = [0.9692, 0.9900, 0.9591, 0.9768, 0.9839]

# Custom threshold metrics
custom_precision = [0.9589, 0.9200, 0.9247, 0.9383, 0.8889]
custom_recall = [0.7778, 0.8313, 0.7611, 0.8352, 0.8333]
custom_f1 = [0.8589, 0.8734, 0.8350, 0.8837, 0.8602]

def summarize(name, values):
    print(f"{name}: {np.mean(values):.4f} ± {np.std(values):.4f}")

print("=== Cross-validation Results (Mean ± Std) ===")
summarize("Accuracy", accuracy)
summarize("Precision", precision)
summarize("Recall", recall)
summarize("F1 Score", f1)
summarize("ROC-AUC", roc_auc)
print("--- Custom Threshold ---")
summarize("Precision", custom_precision)
summarize("Recall", custom_recall)
summarize("F1 Score", custom_f1)

=== Cross-validation Results (Mean ± Std) ===
Accuracy: 0.9995 ± 0.0001
Precision: 0.8766 ± 0.0276
Recall: 0.8206 ± 0.0393
F1 Score: 0.8464 ± 0.0112
ROC-AUC: 0.9758 ± 0.0109
--- Custom Threshold ---
Precision: 0.9262 ± 0.0230
Recall: 0.8077 ± 0.0317
F1 Score: 0.8622 ± 0.0164
